Note: As mentioned in our midterm report, our work on XRay image classification using RNN has been limited on account of inconclusive results. The following code is our preliminary work with this method, the work on which has been long discontinued

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os
import time
import keras
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import *
from sklearn.metrics import *
from tensorflow.keras.models import load_model
import numpy as np
from keras.callbacks import Callback
random.seed(10)
  
train_tfrecord = 'XRay_train.tfrecords'
test_tfrecord = 'XRay_test.tfrecords'


input_path='/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database/'

# Preparing data for generating TFRecords

In [ ]:
def read_directory():
    filenames = []
    labels = []
    covid_files_dir = input_path + 'COVID-19'
    pneumonia_files_dir = input_path + 'Viral Pneumonia'
    normal_files_dir = input_path + 'NORMAL'
    
    for filename in os.listdir(covid_files_dir):
        filenames.append(input_path + 'COVID-19/' + filename)
        labels.append(0)

    for filename in os.listdir(normal_files_dir):
        filenames.append(input_path + 'NORMAL/' + filename)
        labels.append(1)

    for filename in os.listdir(pneumonia_files_dir):
        filenames.append(input_path + 'Viral Pneumonia/' + filename)
        labels.append(2)
        
    labelled_filenames = list(zip(filenames,labels))
    data_size = len(filenames)
    training_data_size = int(data_size * 0.8)
    random.shuffle(labelled_filenames)
    training_list = labelled_filenames[0:training_data_size]
    testing_list = labelled_filenames[training_data_size:]

    training_filenames, training_labels = zip(*training_list)
    testing_filenames, testing_labels = zip(*testing_list)

    return training_filenames, training_labels, testing_filenames, testing_labels

Geenrating TFRecords as per https://www.tensorflow.org/api_docs/python/tf/io/TFRecordWriter[](http://)

In [ ]:
#Images are represented as a sequence of bits and labels as integers resp.
def build_training_tfrecord(training_filenames, training_labels):  
    with tf.io.TFRecordWriter(train_tfrecord)as file_writer:
        for filename, label in zip(training_filenames, training_labels):
            image = open(filename, 'rb').read()
            record_bytes = tf.train.Example(features=tf.train.Features(feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])), 
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  
            })).SerializeToString()
            file_writer.write(record_bytes)
    

def build_testing_tfrecord(test_filenames, testing_labels):  
    with tf.io.TFRecordWriter(test_tfrecord)as file_writer:
        for filename, label in zip(training_filenames, training_labels):
            image = open(filename, 'rb').read()
            record_bytes = tf.train.Example(features=tf.train.Features(feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])), 
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label])) 
            })).SerializeToString()
            file_writer.write(record_bytes)

In [ ]:
def _parse_example(example_string):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_png(feature_dict['image'], channels=1)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [img_size , img_size]) / 255.0
    feature_dict['image'] = tf.squeeze(feature_dict['image']) 
    print(type(feature_dict['image']))
    return feature_dict['image'], feature_dict['label']


In [ ]:
learning_rate = 0.002
img_size = 128
def RNN_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=64, input_shape=(None,img_size), dropout=0.5))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=['accuracy']
    )
    
    return model

In [ ]:
if __name__ == "__main__":
    training_filenames, training_labels, testing_filenames, testing_labels = read_directory()

    build_training_tfrecord(training_filenames, training_labels)
    build_testing_tfrecord(testing_filenames, testing_labels)

    training_dataset = tf.data.TFRecordDataset(train_tfrecord).map(_parse_example)
    testing_dataset = tf.data.TFRecordDataset(test_tfrecord).map(_parse_example)
    
    batch_size = 16
    training_dataset = training_dataset.batch(batch_size)
    testing_dataset = testing_dataset.batch(batch_size)



    print("Training")
    start = time.time()
    model = RNN_model()
    model.summary()
    model.fit(training_dataset,  epochs= 5, verbose = 2)
    model.save('RNNmodel.h5')
    print('Model saved.')
    end = time.time()
    print('Training Time:', end - start)

    print("Testing")
    testing_labels = np.array(testing_labels)
    model = load_model('/kaggle/working/RNNmodel.h5')
    model.evaluate(testing_dataset)
    predIdxs = model.predict(testing_dataset)
    predIdxs = np.argmax(predIdxs, axis=1)
    target_names = ['COVID-19', 'NORMAL', 'Viral Pneumonia']
    print('\n')
    print(classification_report(testing_labels, predIdxs, target_names=target_names, digits=5))


Over the span of 5 epochs, the accuracy barely crosses 0.7. Thus, we have resorted to perfect other models like CNN and SVM that have proven to give much better performance

References:
* https://www.kaggle.com/yichengchen2020/radiography-classification-with-various-models - Idea of using tfrecords 
* https://www.tensorflow.org/api_docs/python/tf/io/TFRecordWriter
* https://www.tensorflow.org/tutorials/load_data/tfrecord
* https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564
* https://towardsdatascience.com/working-with-tfrecords-and-tf-train-example-36d111b3ff4d